In [1]:
# imports
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import glob
import cv2
from sklearn.model_selection import train_test_split
import random
from sklearn.utils import shuffle
from tensorflow.contrib import rnn

In [2]:
class_names = ["EMCI","LMCI","MCI","AD","CN"]
# class_index=[0,1,2,3,4]

In [3]:
path=r"G:\dataset\Alzheimers-ADNI"

In [4]:
'''reading the images'''
data=dict()
# trainData=[]
# trainDataLables=[]
coo_matrix=[]
for name in class_names:
    images = [cv2.imread(file,cv2.COLOR_BGR2GRAY) for file in glob.glob(path+"\\train\\"+name+r'\*.jpg')]
    coo_matrix.append({name:len(images)})
    temp=[]
    for i in images:
        i=cv2.resize(i,(256, 256))
        temp.append(np.asarray(i).reshape(256, 256))
    data[name]=temp


In [5]:
'''balancing imbalanced data'''
print('before')
print(coo_matrix)
max_count=0
for name in class_names:
    if len(data[name])>max_count:
        max_count=len(data[name])
print(max_count)

for name in class_names:
    n=len(data[name])
    for t in range(n,max_count):
        data[name].append(data[name][random.randint(0, n)])

coo_matrix=[]
for name in class_names:
    n=len(data[name])
    coo_matrix.append({name:n})

print('after')
print(coo_matrix)

before
[{'EMCI': 204}, {'LMCI': 61}, {'MCI': 196}, {'AD': 145}, {'CN': 486}]
486
after
[{'EMCI': 486}, {'LMCI': 486}, {'MCI': 486}, {'AD': 486}, {'CN': 486}]


In [6]:
# trainData=[]
# trainDataLables=[]

# for name in class_names:
#     for i in data[name]:
#         trainData.append(i)
#         for i in range(len(class_names)):
#             if name==class_names[i]:
#                 trainDataLables.append(i)

# trainData=np.array(trainData)
# trainDataLables=np.asarray(trainDataLables)
# trainData,trainDataLables=shuffle(trainData,trainDataLables)

trainData=[]
trainDataLables=[]

for name in class_names:
    for i in data[name]:
        trainData.append(i)
        trainDataLables.append(name)

trainData=np.array(trainData)
trainDataLables=np.asarray(trainDataLables)
trainData,trainDataLables=shuffle(trainData,trainDataLables)

In [7]:
lableId=[]
for i in range(len(trainDataLables)):
    for j in range(len(class_names)):
        if trainDataLables[i]==class_names[j]:
            lableId.append(j)

trainLables=tf.keras.utils.to_categorical(lableId, num_classes=len(class_names), dtype='float32')

In [8]:
x_train,x_valid,y_train,y_valid=train_test_split(trainData, trainLables, test_size=0.2, random_state=42)
x_valid,x_test,y_valid,y_test=train_test_split(x_valid, y_valid, test_size=0.5, random_state=42)

In [9]:
# Data Dimension
num_input = 256         # MNIST data input (image shape: 28x28)
timesteps = 256          # Timesteps
n_classes = len(class_names) 

In [10]:
learning_rate = 0.001 # The optimization initial learning rate
epochs = 100           # Total number of training epochs
batch_size = 100      # Training batch size
display_freq = 100    # Frequency of displaying the training results

In [11]:
num_hidden_units = 128  # Number of hidden units of the RNN

In [12]:
# weight and bais wrappers
def weight_variable(shape):
    """
    Create a weight variable with appropriate initialization
    :param name: weight name
    :param shape: weight shape
    :return: initialized weight variable
    """
    initer = tf.truncated_normal_initializer(stddev=0.01)
    return tf.get_variable('W',
                           dtype=tf.float32,
                           shape=shape,
                           initializer=initer)

def bias_variable(shape):
    """
    Create a bias variable with appropriate initialization
    :param name: bias variable name
    :param shape: bias variable shape
    :return: initialized bias variable
    """
    initial = tf.constant(0., shape=shape, dtype=tf.float32)
    return tf.get_variable('b',
                           dtype=tf.float32,
                           initializer=initial)

In [13]:
def RNN(x, weights, biases, timesteps, num_hidden):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, timesteps, 1)

    # Define a rnn cell with tensorflow
    rnn_cell = rnn.BasicRNNCell(num_hidden)

    # Get lstm cell output
    # If no initial_state is provided, dtype must be specified
    # If no initial cell state is provided, they will be initialized to zero
    states_series, current_state = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(current_state, weights) + biases

In [14]:
#  Placeholders for inputs (x) and outputs(y)
x = tf.placeholder(tf.float32, shape=[None, timesteps, num_input], name='X')
y = tf.placeholder(tf.float32, shape=[None, n_classes], name='Y')

In [15]:
# create weight matrix initialized randomely from N~(0, 0.01)
W = weight_variable(shape=[num_hidden_units, n_classes])

# create bias vector initialized as zero
b = bias_variable(shape=[n_classes])

output_logits = RNN(x, W, b, timesteps, num_hidden_units)
y_pred = tf.nn.softmax(output_logits)
 

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [16]:
# Model predictions
cls_prediction = tf.argmax(output_logits, axis=1, name='predictions')

# Define the loss function, optimizer, and accuracy
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=output_logits), name='loss')
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, name='Adam-op').minimize(loss)
correct_prediction = tf.equal(tf.argmax(output_logits, 1), tf.argmax(y, 1), name='correct_pred')
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [17]:
# Creating the op for initializing all variables
init = tf.global_variables_initializer()
 

In [18]:
def randomize(x, y):
    """ Randomizes the order of data samples and their corresponding labels"""
    permutation = np.random.permutation(y.shape[0])
    shuffled_x = x[permutation, :]
    shuffled_y = y[permutation]
    return shuffled_x, shuffled_y

def get_next_batch(x, y, start, end):
    x_batch = x[start:end]
    y_batch = y[start:end]
    return x_batch, y_batch

In [19]:
history=[]
sess = tf.InteractiveSession()
sess.run(init)
global_step = 0
# Number of training iterations in each epoch
num_tr_iter = int(len(y_train) / batch_size)
for epoch in range(epochs):
    print('Training epoch: {}'.format(epoch + 1))
    x_train, y_train = randomize(x_train, y_train)
    for iteration in range(num_tr_iter):
        global_step += 1
        start = iteration * batch_size
        end = (iteration + 1) * batch_size
        x_batch, y_batch = get_next_batch(x_train, y_train, start, end)
        x_batch = x_batch.reshape((batch_size, timesteps, num_input))
        # Run optimization op (backprop)
        feed_dict_batch = {x: x_batch, y: y_batch}
        sess.run(optimizer, feed_dict=feed_dict_batch)

        if iteration % display_freq == 0:
            # Calculate and display the batch loss and accuracy
            loss_batch, acc_batch = sess.run([loss, accuracy],
                                             feed_dict=feed_dict_batch)

            print("iter {0:3d}:\t Loss={1:.2f},\tTraining Accuracy={2:.01%}".
                  format(iteration, loss_batch, acc_batch))

    # Run validation after every epoch

    feed_dict_valid = {x: x_valid[:1000].reshape((-1, timesteps, num_input)), y: y_valid[:1000]}
    loss_valid, acc_valid = sess.run([loss, accuracy], feed_dict=feed_dict_valid)
    print('---------------------------------------------------------')
    print("Epoch: {0}, validation loss: {1:.2f}, validation accuracy: {2:.01%}".
          format(epoch + 1, loss_valid, acc_valid))
    print('---------------------------------------------------------')
    history.append([acc_batch,acc_valid])

Training epoch: 1
iter   0:	 Loss=1.58,	Training Accuracy=31.0%
---------------------------------------------------------
Epoch: 1, validation loss: 1.56, validation accuracy: 35.0%
---------------------------------------------------------
Training epoch: 2
iter   0:	 Loss=1.55,	Training Accuracy=38.0%
---------------------------------------------------------
Epoch: 2, validation loss: 1.50, validation accuracy: 35.0%
---------------------------------------------------------
Training epoch: 3
iter   0:	 Loss=1.46,	Training Accuracy=42.0%
---------------------------------------------------------
Epoch: 3, validation loss: 1.44, validation accuracy: 38.7%
---------------------------------------------------------
Training epoch: 4
iter   0:	 Loss=1.37,	Training Accuracy=44.0%
---------------------------------------------------------
Epoch: 4, validation loss: 1.38, validation accuracy: 43.2%
---------------------------------------------------------
Training epoch: 5
iter   0:	 Loss=1.24,	

In [30]:
history=np.array(history).transpose()
index=0
for i in range(len(history[1])):
    if history[1][i]==max(history[1]):
        index=i

print("traing acc:",history[0][index],"| validation acc:",history[1][index])

traing acc: 0.88 | validation acc: 0.76543206


In [34]:
testData=[]
testDataLables=[]
for name in class_names:
    images = [cv2.imread(file,cv2.COLOR_BGR2GRAY) for file in glob.glob(path+"\\test\\"+name+r'\*.jpg')]
    for i in images:
        i=cv2.resize(i,(256, 256))
        testData.append(np.array(i).reshape((256, 256)))
        testDataLables.append(name)


testData=np.asarray(testData)
testDataLables=np.asarray(testDataLables)



testlableId=[]
for i in range(len(testDataLables)):
    for j in range(len(class_names)):
        if testDataLables[i]==class_names[j]:
            testlableId.append(j)

testLables=tf.keras.utils.to_categorical(testlableId, num_classes=len(class_names), dtype='float32')

In [35]:
feed_dict_valid = {x: testData, y: testLables}
loss_valid, acc_valid = sess.run([loss, accuracy], feed_dict=feed_dict_valid)
print(acc_valid)

0.30769232
